In [1]:
import pandas as pd
import numpy as np

# Simulate data for 100 stocks over 36 months
np.random.seed(42)
n_stocks = 100
n_months = 36
dates = pd.date_range('2018-01-31', periods=n_months, freq='M')
stocks = [f'STK{i:03d}' for i in range(n_stocks)]

index = pd.MultiIndex.from_product([dates, stocks], names=["date", "stock"])
df = pd.DataFrame(index=index).reset_index()
df

,date,stock
0,2018-01-31,STK000
1,2018-01-31,STK001
2,2018-01-31,STK002
3,2018-01-31,STK003
4,2018-01-31,STK004
...,...,...
3595,2020-12-31,STK095
3596,2020-12-31,STK096
3597,2020-12-31,STK097
3598,2020-12-31,STK098


In [2]:
# Simulate market cap and book equity
df['market_cap'] = np.random.lognormal(mean=10, sigma=1, size=len(df))
df['book_equity'] = df['market_cap'] / np.random.uniform(1.1, 2.0, len(df))

# Simulate next-month return
df['next_return'] = np.random.normal(0.01, 0.05, size=len(df))

# Add fiscal year (for BE) and sorting year (June sort)
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['sort_year'] = df['date'].apply(lambda d: d.year if d.month >= 7 else d.year - 1)


In [3]:
df

,date,stock,market_cap,book_equity,next_return,year,month,sort_year
0,2018-01-31,STK000,36196.371320,27203.609760,0.016760,2018,1,2017
1,2018-01-31,STK001,19182.155002,14771.571298,-0.006312,2018,1,2017
2,2018-01-31,STK002,42095.180602,34382.701745,-0.057267,2018,1,2017
3,2018-01-31,STK003,101015.561318,53926.718951,-0.004058,2018,1,2017
4,2018-01-31,STK004,17428.230714,11243.639467,0.035267,2018,1,2017
...,...,...,...,...,...,...,...,...
3595,2020-12-31,STK095,69316.681493,57098.217603,0.005843,2020,12,2020
3596,2020-12-31,STK096,8321.694359,5628.215129,0.051171,2020,12,2020
3597,2020-12-31,STK097,60302.771075,39340.360440,0.057282,2020,12,2020
3598,2020-12-31,STK098,30826.961611,21427.244418,0.035218,2020,12,2020


In [4]:
# Create June snapshots for sorting
june_snapshots = df[df['month'] == 6].copy()

# Create size breakpoints (NYSE median simulated as median of all)
breakpoints = june_snapshots.groupby('sort_year')['market_cap'].median().rename("size_median")
june_snapshots = june_snapshots.merge(breakpoints, on='sort_year')
june_snapshots['size'] = np.where(june_snapshots['market_cap'] <= june_snapshots['size_median'], 'S', 'B')
# Compute B/M ratio
june_snapshots['bm'] = june_snapshots['book_equity'] / june_snapshots['market_cap']
june_snapshots

,date,stock,market_cap,book_equity,next_return,year,month,sort_year,size_median,size,bm
0,2018-06-30,STK000,55613.293730,36413.004122,0.053948,2018,6,2017,19734.518906,B,0.654754
1,2018-06-30,STK001,148659.931942,105981.949075,0.034030,2018,6,2017,19734.518906,B,0.712915
2,2018-06-30,STK002,5439.445618,2820.583641,0.082328,2018,6,2017,19734.518906,S,0.518542
3,2018-06-30,STK003,38675.795214,34817.879323,-0.012993,2018,6,2017,19734.518906,B,0.900250
4,2018-06-30,STK004,11491.437030,7193.237472,-0.011482,2018,6,2017,19734.518906,S,0.625965
...,...,...,...,...,...,...,...,...,...,...,...
295,2020-06-30,STK095,23800.943900,14021.897705,0.111341,2020,6,2019,23893.278191,S,0.589132
296,2020-06-30,STK096,28502.655579,23711.737324,0.053138,2020,6,2019,23893.278191,B,0.831913
297,2020-06-30,STK097,6362.899346,3638.947919,-0.031504,2020,6,2019,23893.278191,S,0.571901
298,2020-06-30,STK098,30766.337017,19845.733271,-0.004092,2020,6,2019,23893.278191,B,0.645047


In [5]:
# Compute B/M breakpoints using simulated NYSE stocks (here: all)
bm_30 = june_snapshots.groupby('sort_year')['bm'].quantile(0.3).rename("bm30")
bm_70 = june_snapshots.groupby('sort_year')['bm'].quantile(0.7).rename("bm70")
june_snapshots = june_snapshots.merge(bm_30, on='sort_year').merge(bm_70, on='sort_year')


In [6]:
def assign_bm_group(row):
    if row['bm'] <= row['bm30']:
        return 'L'
    elif row['bm'] <= row['bm70']:
        return 'M'
    else:
        return 'H'

In [7]:
june_snapshots['bm_group'] = june_snapshots.apply(assign_bm_group, axis=1)

# Assign each stock to one of 6 portfolios
june_snapshots['portfolio'] = june_snapshots['size'] + '/' + june_snapshots['bm_group']
june_snapshots

,date,stock,market_cap,book_equity,next_return,year,month,sort_year,size_median,size,bm,bm30,bm70,bm_group,portfolio
0,2018-06-30,STK000,55613.293730,36413.004122,0.053948,2018,6,2017,19734.518906,B,0.654754,0.603568,0.737574,M,B/M
1,2018-06-30,STK001,148659.931942,105981.949075,0.034030,2018,6,2017,19734.518906,B,0.712915,0.603568,0.737574,M,B/M
2,2018-06-30,STK002,5439.445618,2820.583641,0.082328,2018,6,2017,19734.518906,S,0.518542,0.603568,0.737574,L,S/L
3,2018-06-30,STK003,38675.795214,34817.879323,-0.012993,2018,6,2017,19734.518906,B,0.900250,0.603568,0.737574,H,B/H
4,2018-06-30,STK004,11491.437030,7193.237472,-0.011482,2018,6,2017,19734.518906,S,0.625965,0.603568,0.737574,M,S/M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,2020-06-30,STK095,23800.943900,14021.897705,0.111341,2020,6,2019,23893.278191,S,0.589132,0.587877,0.705062,M,S/M
296,2020-06-30,STK096,28502.655579,23711.737324,0.053138,2020,6,2019,23893.278191,B,0.831913,0.587877,0.705062,H,B/H
297,2020-06-30,STK097,6362.899346,3638.947919,-0.031504,2020,6,2019,23893.278191,S,0.571901,0.587877,0.705062,L,S/L
298,2020-06-30,STK098,30766.337017,19845.733271,-0.004092,2020,6,2019,23893.278191,B,0.645047,0.587877,0.705062,M,B/M


In [8]:

# Assign portfolios to main DataFrame for next 12 months
df = df.merge(june_snapshots[['date', 'stock', 'portfolio']], on=['stock', 'date'], how='left')

# Forward fill portfolio assignment (simulate 12-month holding)
df['portfolio'] = df.groupby('stock')['portfolio'].ffill(limit=12)



In [9]:
df

,date,stock,market_cap,book_equity,next_return,year,month,sort_year,portfolio
0,2018-01-31,STK000,36196.371320,27203.609760,0.016760,2018,1,2017,NaN
1,2018-01-31,STK001,19182.155002,14771.571298,-0.006312,2018,1,2017,NaN
2,2018-01-31,STK002,42095.180602,34382.701745,-0.057267,2018,1,2017,NaN
3,2018-01-31,STK003,101015.561318,53926.718951,-0.004058,2018,1,2017,NaN
4,2018-01-31,STK004,17428.230714,11243.639467,0.035267,2018,1,2017,NaN
...,...,...,...,...,...,...,...,...,...
3595,2020-12-31,STK095,69316.681493,57098.217603,0.005843,2020,12,2020,S/M
3596,2020-12-31,STK096,8321.694359,5628.215129,0.051171,2020,12,2020,B/H
3597,2020-12-31,STK097,60302.771075,39340.360440,0.057282,2020,12,2020,S/L
3598,2020-12-31,STK098,30826.961611,21427.244418,0.035218,2020,12,2020,B/M


In [10]:
# Compute monthly value-weighted returns for each portfolio
df['weight'] = df.groupby(['date', 'portfolio'])['market_cap'].transform(lambda x: x / x.sum())
df['port_return'] = df['weight'] * df['next_return']

# Drop rows without a portfolio assignment
df_clean = df.dropna(subset=['portfolio'])
df_clean

,date,stock,market_cap,book_equity,next_return,year,month,sort_year,portfolio,weight,port_return
500,2018-06-30,STK000,55613.293730,36413.004122,0.053948,2018,6,2017,B/M,0.059108,0.003189
501,2018-06-30,STK001,148659.931942,105981.949075,0.034030,2018,6,2017,B/M,0.158003,0.005377
502,2018-06-30,STK002,5439.445618,2820.583641,0.082328,2018,6,2017,S/L,0.031559,0.002598
503,2018-06-30,STK003,38675.795214,34817.879323,-0.012993,2018,6,2017,B/H,0.044097,-0.000573
504,2018-06-30,STK004,11491.437030,7193.237472,-0.011482,2018,6,2017,S/M,0.049366,-0.000567
...,...,...,...,...,...,...,...,...,...,...,...
3595,2020-12-31,STK095,69316.681493,57098.217603,0.005843,2020,12,2020,S/M,0.089169,0.000521
3596,2020-12-31,STK096,8321.694359,5628.215129,0.051171,2020,12,2020,B/H,0.009762,0.000500
3597,2020-12-31,STK097,60302.771075,39340.360440,0.057282,2020,12,2020,S/L,0.072372,0.004146
3598,2020-12-31,STK098,30826.961611,21427.244418,0.035218,2020,12,2020,B/M,0.039153,0.001379


In [19]:
# Aggregate portfolio returns
portfolio_returns = df_clean.groupby(['date', 'portfolio'])['port_return'].sum().unstack()
portfolio_returns


portfolio,B/H,B/L,B/M,S/H,S/L,S/M
date,,,,,,
2018-06-30,0.006346,0.014491,0.021888,0.036860,0.019172,0.001445
2018-07-31,-0.002644,0.007796,0.020102,0.009941,-0.023474,-0.006829
2018-08-31,0.009754,0.002449,0.022505,0.025291,-0.031507,-0.014116
2018-09-30,0.019501,-0.033851,0.014782,0.008532,0.041423,0.005301
2018-10-31,-0.006187,0.016201,0.015598,0.027728,0.020450,-0.006059
2018-11-30,0.009835,0.001863,-0.012422,0.026183,0.006617,0.019724
2018-12-31,0.006750,0.008931,0.004768,-0.006816,-0.006143,0.022811
2019-01-31,-0.004269,0.005637,0.016997,-0.002598,-0.030106,0.040480
2019-02-28,0.015221,0.002893,0.016664,-0.011894,0.044784,0.026313


In [20]:

# Compute HML: value (high B/M) minus growth (low B/M)
portfolio_returns['HML'] = 0.5 * (portfolio_returns['S/H'] + portfolio_returns['B/H']) - \
                           0.5 * (portfolio_returns['S/L'] + portfolio_returns['B/L'])

# Compute SMB: small minus big
portfolio_returns['SMB'] = (1/3) * (portfolio_returns['S/L'] + portfolio_returns['S/M'] + portfolio_returns['S/H']) - \
                           (1/3) * (portfolio_returns['B/L'] + portfolio_returns['B/M'] + portfolio_returns['B/H'])

# Compute MKT: market return as weighted average of all 6 portfolios (proxy)
# This is not exact CRSP VWRETD, but a reasonable approximation
portfolio_returns['MKT'] = portfolio_returns[[f'S/{g}' for g in ['L', 'M', 'H']] + [f'B/{g}' for g in ['L', 'M', 'H']]].mean(axis=1)

# Show results
factors = portfolio_returns[['MKT', 'SMB', 'HML']]


In [21]:
factors

portfolio,MKT,SMB,HML
date,,,
2018-06-30,0.016700,0.004917,0.004772
2018-07-31,0.000815,-0.015205,0.011488
2018-08-31,0.002396,-0.018347,0.032052
2018-09-30,0.009281,0.018275,0.010230
2018-10-31,0.011288,0.005503,-0.007555
2018-11-30,0.008633,0.017749,0.013769
2018-12-31,0.005050,-0.003533,-0.001427
2019-01-31,0.004357,-0.003530,0.008801
2019-02-28,0.015664,0.008142,-0.022175
